In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import numpy as np
from shapely.ops import unary_union

# --- 1. Load Data ---
# Ensure the file paths below are correct for your system.

# Municipalities Shapefile
shapefile_path_municipalities = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\MT_Municipios_2022\\MT_Municipios_2022.shp"
municipalities = gpd.read_file(shapefile_path_municipalities)

# Highways Shapefile
shapefile_path_highways = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\Rodovias\\SNV_202410A.shp"
highways = gpd.read_file(shapefile_path_highways)

# Colonization Data from Excel
file_path_excel = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Dissertação\\Quadros e gráficos\\Table 1.xlsx"
colonization = pd.read_excel(file_path_excel, sheet_name='Planilha1')


# --- 2. Process and Merge Data ---

# Filter highways for Mato Grosso (MT)
filtered_highways_MT = highways[highways['sg_uf'] == 'MT']

# Group highways by their BR number and combine their geometries into single lines
def combine_geometries(group):
    return unary_union(group['geometry'])

combined_geometries_series = filtered_highways_MT.groupby('vl_br').apply(combine_geometries)
combined_geometries = gpd.GeoDataFrame(geometry=combined_geometries_series.values, crs=filtered_highways_MT.crs)
combined_geometries['vl_br'] = combined_geometries_series.index

# Join municipal data with colonization data
merged = pd.merge(municipalities, colonization, left_on='NM_MUN', right_on='City', how='inner')

# Filter for cities that have a valid colonization year
colonization_cities = merged[merged['Year'].notna()].copy()


# --- 3. Assign Unique, High-Contrast Colors ---

# A list of 32 distinct, high-contrast colors.
high_contrast_colors = [
    '#FF5733', '#33FF57', '#3357FF', '#FF33A1', '#A133FF', '#33FFA1',
    '#FFC300', '#C70039', '#900C3F', '#581845', '#00FFFF', '#FF00FF',
    '#00FF00', '#FFFF00', '#0000FF', '#FF7F50', '#DC143C', '#008B8B',
    '#B8860B', '#006400', '#8B008B', '#FF8C00', '#E9967A', '#8FBC8F',
    '#483D8B', '#2F4F4F', '#9400D3', '#FF1493', '#1E90FF', '#ADFF2F',
    '#DA70D6', '#FFD700'
]

# Get unique highways and cities to assign colors
unique_highways = sorted(combined_geometries['vl_br'].unique())
unique_cities = sorted(colonization_cities['NM_MUN'].unique())

# Check if there are enough colors for all unique items
total_unique_items = len(unique_highways) + len(unique_cities)
if total_unique_items > len(high_contrast_colors):
    print(f"Warning: There are {total_unique_items} unique items but only {len(high_contrast_colors)} colors.")
    # Extend the color list by cycling to avoid crashing
    high_contrast_colors = high_contrast_colors * (total_unique_items // len(high_contrast_colors) + 1)

# Create color mappings and add a 'Color' column to each GeoDataFrame
color_iterator = iter(high_contrast_colors)

highway_color_map = {br: next(color_iterator) for br in unique_highways}
city_color_map = {city: next(color_iterator) for city in unique_cities}

combined_geometries['Color'] = combined_geometries['vl_br'].map(highway_color_map)
colonization_cities['Color'] = colonization_cities['NM_MUN'].map(city_color_map)


# --- 4. Plotting the Map ---

# Adjust figsize to give more vertical space for the legends
fig, ax = plt.subplots(figsize=(12, 14))

# Plot the base map of all municipalities in gray
municipalities.plot(ax=ax, color="#ffffff", edgecolor='black', linewidth=0.5)

# Plot the colored colonization city polygons
colonization_cities.plot(ax=ax, color=colonization_cities['Color'], edgecolor='black', linewidth=0.7)

# Plot the highways on top, using the new 'Color' column
combined_geometries.plot(ax=ax, color=combined_geometries['Color'], linewidth=2.5)


# --- 5. Create and Position Legends ---

# Create legend handles for highways
legend_elements_highways = [
    Line2D([0], [0], color=color, lw=2, label=f'BR-{br}')
    for br, color in highway_color_map.items()
]

# Create legend handles for cities (using a patch to represent the fill)
legend_elements_colonization = [
    Patch(facecolor=color, edgecolor='black', linewidth=0.7, label=city)
    for city, color in city_color_map.items()
]

# Create the first legend for Highways
leg1 = ax.legend(
    handles=legend_elements_highways,
    title="Highways",
    loc='upper center',
    bbox_to_anchor=(0.5, -0.05), # Position below the plot
    ncol=8, # Arrange in 8 columns
    prop={'size': 9},
    fancybox=True,
)

# Add the first legend as an artist to prevent it from being overwritten
ax.add_artist(leg1)

# Create the second legend for Colonization Cities, positioned below the first
ax.legend(
    handles=legend_elements_colonization,
    title="Colonization cities",
    loc='upper center',
    bbox_to_anchor=(0.5, -0.15), # Adjust y-value to stack below the first legend
    ncol=8,
    prop={'size': 9},
    fancybox=True
)


# Clean up the plot appearance
ax.set_axis_off()
# Adjust subplot parameters to make room for the legends at the bottom
fig.subplots_adjust(bottom=0.2)


# Save the figure with high resolution, ensuring the full legend is included
output_path = r"C:\Users\daves\OneDrive\Pessoal\Acadêmico\Mestrado\Dissertação - Execução\Dissertação\Quadros e gráficos\Graph-Colonization with Highways.jpg"
plt.savefig(output_path, dpi=1800, bbox_inches='tight')

# Display the plot
plt.show()

In [ ]:
import folium

# Create a Folium map centered on Mato Grosso
m = folium.Map(location=[-12.6819, -56.9211], zoom_start=6, tiles='CartoDB positron')

# Add municipalities (light gray borders)
folium.GeoJson(
    municipalities,
    style_function=lambda x: {'fillColor': 'white', 'color': 'black', 'weight': 0.5, 'fillOpacity': 0.3},
    name='Municipalities'
).add_to(m)

# Add highways (colored by BR number)
import matplotlib.cm as cm
import matplotlib.colors as colors

# Generate colors for each BR highway
vl_br_unique = combined_geometries['vl_br'].unique()
cmap = cm.get_cmap('tab20', len(vl_br_unique))  # Use the same colormap as before
norm = colors.Normalize(vmin=0, vmax=len(vl_br_unique)-1)

for idx, row in combined_geometries.iterrows():
    br_num = row['vl_br']
    color = colors.rgb2hex(cmap(norm(idx)))  # Convert matplotlib color to hex
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x, color=color: {'color': color, 'weight': 3},
        tooltip=f'BR-{br_num}'
    ).add_to(m)

# URL for Mato Grosso flag
bandeira_mt_url = "https://upload.wikimedia.org/wikipedia/commons/0/0b/Bandeira_de_Mato_Grosso.svg"

# Add colonization cities (using folium.Marker with CustomIcon)
for idx, row in colonization_cities.iterrows():
    year_display = int(row['Year']) if pd.notna(row['Year']) else 'N/A'

    # Criar o ícone customizado com a bandeira
    custom_icon = folium.CustomIcon(
        icon_image=bandeira_mt_url,
        icon_size=(15, 10.5),  # Icon in e pixels (width, height)
                               # Proportion is 10:7
        icon_anchor=(7.5, 5.25),
        popup_anchor=(0, -10)
    )

    folium.Marker(
        location=[row.geometry.centroid.y, row.geometry.centroid.x],
        tooltip=f"{row['NM_MUN']} (Year: {year_display})",
        popup=f"<b>{row['NM_MUN']}</b><br>Projeto de Colonização<br>Year: {year_display}",
        icon=custom_icon
    ).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save to HTML
output_path = r"C:\Users\daves\OneDrive\Pessoal\Acadêmico\Mestrado\Dissertação - Execução\Dissertação\Quadros e gráficos\Colonization_with_Highways.html"
m.save(output_path)

# Display in notebook (if using Jupyter)
m